In [34]:

from IPython.display import clear_output
from ipaddress import ip_address, ip_network, IPv4Address, IPv6Address
from enum import Enum

import panel as pn
import ipywidgets as widgets
import pandas as pd
import altair as alt
import pathlib
import datetime
import os

pn.extension('vega')

SRC_AND_DST_IP = ["SOURCE IP ADDRESS", "DESTINATION IP ADDRESS"]

SRC_IP = SRC_AND_DST_IP[0]
DST_IP = SRC_AND_DST_IP[1]

# DATA_TYPES = Enum("data_type", names={"": "BYTES", "Packets": "PACKETS",
#                  "Flows": "FLOWS", "Subdomains of muni.cz": "DNS QNAME",
#                  "Unusual TLDs": "UNUSUAL TLD", 
#                  "IP requesting unusual TLD": "IP REQUESTING UNUSUAL TLD",
#                  "Most DNS requests": "MOST DNS REQUESTS", 
#                  'Duration with dst IP': "DURATION"})

DATA_TYPE_DSC = {"Bytes": "BYTES", "Packets": "PACKETS",
                 "Flows": "FLOWS", "Subdomains of muni.cz": "DNS QNAME",
                 "Unusual TLDs": "UNUSUAL TLD", 
                 "IP requesting unusual TLD": "IP REQUESTING UNUSUAL TLD",
                 "Most DNS requests": "MOST DNS REQUESTS", 
                 'Duration with dst IP': "DURATION", 
                 "DNS communication": "DNS_COMM", "SMTP communication": "SMTP_COMM"}

SINGLE_PAIR_DSC = {"Pair" : "PAIR",
                   "Single": "SINGLE"}

COUNT = "COUNT"

file_path = os.path.join("Task 1", "20240701_16-17_100l.csv")
dir_path = pathlib.Path().resolve()

csv_file = pd.read_csv(dir_path / file_path)
flow_data: pd.DataFrame = pd.DataFrame(csv_file)

def get_first_time_flow_string():
    return flow_data["START TIME - FIRST SEEN"].min()

def get_last_time_flow_string():
    return flow_data["START TIME - FIRST SEEN"].max()

def get_first_time_flow():
    return datetime.datetime.fromisoformat(get_first_time_flow_string())

def get_last_time_flow():
    return datetime.datetime.fromisoformat(get_last_time_flow_string())


def ipv4_in_range_cmp(ip, min, max):
    if is_ipv4(ip):
        return min <= try_convert_ip_to_int(ip) <= max
    return True

def ipv6_in_range_cmp(ip, min, max):
    if is_ipv6(ip):
        return min <= try_convert_ip_to_int(ip) <= max
    return True    

def try_convert_ip_to_int(ip):
    try:
        return int(ip_address(ip))
    except Exception:
        return 0

def is_ipv6(addr):
    try:
        return type(ip_address(addr)) is IPv6Address
    except Exception:
        return False

def is_ipv4(addr):
    try:
        return type(ip_address(addr)) is IPv4Address
    except Exception:
        return False

def to_datetime(x):
    return datetime.datetime.fromisoformat(x)

def time_to_seconds(time_str):
    time_parts = time_str.split()
    minutes = int(time_parts[0].strip('m'))
    seconds = int(time_parts[1].strip('s'))

    total_minutes = minutes * 60 + seconds
    return total_minutes

def is_in_muni_subdomain(input_str):
    return "muni.cz" in input_str

style = {'description_width': 'initial'}

apply_changes_widget = widgets.Button(
    description='Apply changes',
    tooltip='Apply changes',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    style = style,
)

top_n_widget = widgets.Dropdown(
    options=['10', '20', '30', '40'],
    value='10',
    description='Top N stats',
    disabled=False,
    style = style,
)

single_or_pair_widget = widgets.ToggleButtons(
    options=['Single', 'Pair'],
    description='Info about single IP addresss or pair:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    style = style,
)

src_dst_widget = widgets.ToggleButtons(
    options=['Received', 'Sent'],
    description='Direction:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    style=style,
)

data_type_widget = widgets.ToggleButtons(
    options=list(DATA_TYPE_DSC.keys()),
    description='flow_data type:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    style = {'button_width':'200px'}, 
    layout={'width': '520px'},
)

period_widget = widgets.IntRangeSlider(
    min=0,
    max=36000,
    value=[0, 36000],
    step=5,
    description="Time range:",
    disabled=False,
    style = style,
)

ipv4_range_widget = widgets.Textarea(
    value='0.0.0.0/0,',
    description='List of IPv4 masks, separeted by \",\"',
    disabled=False,
    style = style,
    layout = widgets.Layout(width='50%')
)

ipv6_range_widget = widgets.Textarea(
    value='0:0:0:0:0:0:0:0/0,',
    description='List of IPv6 masks, separeted by \",\"',
    disabled=False,
    style = style,
    layout = widgets.Layout(width='80%')
)

protocol_widget = widgets.SelectMultiple(
    options=["ALL", "TCP", "UDP", "SMTP",
             "DNS", "HTTP", "HTTPS",
             "SMB", "TELNET",
             "RDP", "FTP", "TFTFP"
            ],
    value=("ALL",),
    description='Protocol',
    disabled=False,
    style = style,
)

port_widget = widgets.Dropdown(
    options=["ALL",
             "8080, 8888, 591, 82",
             "8443, 9443, 4443",
             "5353, 5355",
             "2121, 8021",
             "2525, 26",
             "3390, 3391, 4000",
            ],
    description='Ports',
    disabled=False,
    style = style,
    layout={'Height': '520px'}
)

minimal_duration_widget = widgets.Text(
    value='disabled',
    description='Minimal duration of a flow in seconds',
    disabled=True,
    style = style,
    layout = widgets.Layout(width='50%')
)

table_widget = widgets.HTML(value=flow_data.iloc[:0].to_html())

ui_widgets = [
              top_n_widget, single_or_pair_widget, src_dst_widget, data_type_widget,
              minimal_duration_widget, period_widget, ipv4_range_widget, ipv6_range_widget,
              protocol_widget, port_widget, apply_changes_widget,
              ]

def display_widgets():
    for ui_widget in ui_widgets:
        display(ui_widget)

def disable_button(button):
    button.button_style = ''
    button.disabled = True

def enable_button(button):
    button.button_style = 'info'
    button.disabled = False

def process_flow_pair(curr_data, top_n_n):
    tmp_data = curr_data[[SRC_IP, DST_IP]].apply(lambda x: tuple(sorted(x)), axis=1).value_counts().reset_index()
    tmp_data.columns = ['pair', DATA_TYPE_DSC["Flows"]]
    tmp_data[[SRC_IP, DST_IP]] = pd.DataFrame(tmp_data['pair'].tolist(), index=tmp_data.index)
    processed_data = tmp_data.drop(columns=['pair']).nlargest(top_n_n, columns=DATA_TYPE_DSC["Flows"])
    processed_data['src-dst'] = processed_data[SRC_IP] + ' - ' + processed_data[DST_IP]
    print(processed_data)
    return processed_data[['src-dst', DATA_TYPE_DSC["Flows"]]]

def process_duration_single(curr_data, g_data_type, minimal_duration, top_n_n):
    processed_data = pd.DataFrame(curr_data)
    processed_data[g_data_type] = processed_data[g_data_type].apply(time_to_seconds)
    return processed_data[(processed_data[g_data_type] >= int(minimal_duration))].groupby(DST_IP).agg(
                    DURATION=(g_data_type, 'sum'),
                    COUNT=(DST_IP, 'count'),
                    ).reset_index().nlargest(top_n_n, columns=g_data_type)

def process_ip_requesting_unusual_tld(curr_data, g_data_type, top_n_n, unusual_tlds_regex):
    processed_data = curr_data[curr_data[g_data_type].str.contains(unusual_tlds_regex, regex=True, na=False)]
    return processed_data.groupby(SRC_IP)[SRC_IP].count().reset_index(name="IP REQUESTING UNUSUAL TLD").nlargest(top_n_n, columns="IP REQUESTING UNUSUAL TLD")
      
def process_unusual_tld(curr_data, g_data_type, top_n_n, unusual_tlds_regex):
    processed_data = curr_data[curr_data[g_data_type].str.contains(unusual_tlds_regex, regex=True, na=False)]
    processed_data = processed_data.groupby(g_data_type)[g_data_type].count().reset_index(name=COUNT).nlargest(top_n_n, columns=COUNT)
    return processed_data.rename(columns={g_data_type: "UNUSUAL TLD"})

def process_by_type(curr_data, g_src_dst, g_data_type, top_n_n, single_or_pair, minimal_duration):
    if curr_data.empty:
        return curr_data

    unusual_tlds_regex = "\.xyz$|\.top$|\.club$|\.site$|\.online$|\.loan$|\.trade$|\.accountants$|\.space$"
    if single_or_pair == "Pair":
        tmp_data = pd.DataFrame(curr_data)
        if g_data_type == "FLOWS":
            processed_data = process_flow_pair(curr_data, top_n_n)
        else:
            tmp_data['pair'] = curr_data[[SRC_IP, DST_IP]].apply(lambda x: tuple(sorted(x)), axis=1)
            processed_data = tmp_data.groupby('pair', as_index=False)[g_data_type].sum().nlargest(top_n_n, columns=g_data_type)

    else:
        if g_data_type == "FLOWS":
            processed_data = curr_data.groupby(g_src_dst)[g_src_dst].count().reset_index(name=g_data_type).nlargest(top_n_n, columns=g_data_type)
        elif g_data_type == "DURATION":
            processed_data = process_duration_single(curr_data, g_data_type, minimal_duration, top_n_n)
        elif g_data_type == "DNS QNAME":
            processed_data = curr_data[curr_data[g_data_type].str.contains('muni.cz', na=False)]
            processed_data = processed_data.groupby(g_data_type)[g_data_type].count().reset_index(name=COUNT).nlargest(top_n_n, columns=COUNT)
        elif g_data_type == "UNUSUAL TLD":
            processed_data = process_unusual_tld(curr_data, "DNS QNAME", top_n_n, unusual_tlds_regex)
        elif g_data_type == "IP REQUESTING UNUSUAL TLD":
            processed_data = process_ip_requesting_unusual_tld(curr_data, "DNS QNAME", top_n_n, unusual_tlds_regex)
        elif g_data_type == "MOST DNS REQUESTS":
            processed_data = curr_data[curr_data['DNS QNAME'].notna()].groupby(SRC_IP)[SRC_IP].count().reset_index(name=g_data_type).nlargest(top_n_n, columns=g_data_type)
        elif g_data_type == "SMTP_COMM":
            processed_data = process_by_dns_smtp(curr_data, "SMTP")
        elif g_data_type == "DNS_COMM":
            processed_data = process_by_dns_smtp(curr_data, "DNS")
        else:
            processed_data = curr_data[[g_src_dst, g_data_type]].groupby(g_src_dst).sum().reset_index().nlargest(top_n_n, columns=g_data_type)
            processed_data = processed_data.merge(flow_data.groupby(g_src_dst)[g_src_dst].count().reset_index(name=COUNT), how='left', on=g_src_dst)

    return processed_data

def process_by_ipvx_range_pair(curr_data, ip_ranges, cmp_func):
    tmp_data = pd.DataFrame()
    for ip_subnet in ip_ranges:
        if len(ip_subnet) <= 1:
            continue
        ip_range_int = ip_network(ip_subnet.strip())
        ip_start, ip_end = int(ip_range_int[0]), int(ip_range_int[-1])
    
        tmp_data = pd.concat([tmp_data, curr_data[(curr_data[SRC_IP].apply(cmp_func, args=(ip_start, ip_end))) &
                                                    (curr_data[DST_IP].apply(cmp_func, args=(ip_start, ip_end)))]])
    
    return tmp_data

#need to fix filtering ipv4 and ipv6 addresses separately
def process_by_ipvx_range_single(curr_data, g_src_dst, ip_ranges):
    tmp_data = pd.DataFrame()
    for ip_subnet in ip_ranges:
        if len(ip_subnet) <= 1:
            continue
        ip_range_int = ip_network(ip_subnet.strip())
        ip_start, ip_end = int(ip_range_int[0]), int(ip_range_int[-1])

        tmp_data = pd.concat([tmp_data, curr_data[(curr_data[g_src_dst].apply(try_convert_ip_to_int).between(ip_start, ip_end))]])

    return pd.DataFrame(tmp_data)

def process_by_ip_range(curr_data, g_src_dst, ipv4_ranges_str, ipv6_ranges_str, single_or_pair):
    if curr_data.empty:
        return curr_data

    if single_or_pair == 'Single':
        ipv4_addrs = curr_data[(curr_data[g_src_dst].apply(is_ipv4))]
        ipv6_addrs = curr_data[(curr_data[g_src_dst].apply(is_ipv6))]
        return pd.concat([process_by_ipvx_range_single(ipv4_addrs, g_src_dst, ipv4_ranges_str), 
                        process_by_ipvx_range_single(ipv6_addrs, g_src_dst, ipv6_ranges_str),
                        ])

    ipv4_addrs = curr_data[(curr_data[SRC_IP].apply(is_ipv4)) | (curr_data[DST_IP].apply(is_ipv4))]
    ipv6_addrs = curr_data[(curr_data[SRC_IP].apply(is_ipv6)) | (curr_data[DST_IP].apply(is_ipv6))]

    return pd.concat([process_by_ipvx_range_pair(ipv4_addrs, ipv4_ranges_str, ipv4_in_range_cmp), 
                      process_by_ipvx_range_pair(ipv6_addrs, ipv6_ranges_str, ipv6_in_range_cmp)])

def process_by_time_range(curr_data, time_period):
    if curr_data.empty:
        return curr_data
    start_offset, end_offset = time_period

    start = get_first_time_flow() + datetime.timedelta(minutes=start_offset)
    end = get_first_time_flow() + datetime.timedelta(minutes=end_offset)

    return curr_data[(curr_data["START TIME - FIRST SEEN"].apply(to_datetime) >= start) &
                      ((curr_data["START TIME - FIRST SEEN"].apply(to_datetime)) <= end)]

def process_by_dns_smtp(curr_data, protocol):
    if protocol == "DNS":
        protocol_regex = "?!dns"
    else:
        protocol_regex = "?!smtp"

    processed_data = curr_data[curr_data["DNS QNAME"].str.contains(protocol_regex, regex=True, na=False)]
    return processed_data

def process_by_protocol(curr_data, protocols):
    if curr_data.empty:
        return curr_data
    print(protocols)
    if "ALL" in protocols:
        return curr_data

    processed_data = pd.DataFrame()
    for protocol in protocols:
        if protocol in ["TCP", "UDP"]:
            processed_data = pd.concat([processed_data, curr_data[curr_data["PROTOCOL"] == protocol]])
        else:
            processed_data = pd.concat([processed_data, curr_data[curr_data["DETECTED PROTOCOL"] == protocol]])

    return processed_data

def process_by_port(curr_data, ports, g_src_dst):
    if curr_data.empty:
        return curr_data

    if ports == "ALL":
        return curr_data

    port_dir = "SOURCE PORT" if g_src_dst == SRC_IP else "DESTINATION PORT"

    tmp_data = pd.DataFrame()
    ports_list = ports.split(', ')

    for port in ports_list:
        tmp_data = pd.concat([tmp_data, curr_data[curr_data[port_dir] == int(port)]])

    return tmp_data

def get_traffic_data(g_src_dst, g_data_type, top_n, time_period,
                     ipv4_range_str, ipv6_range_str, single_or_pair,
                     protocol, ports, minimal_duration):
    processed_data = pd.DataFrame(flow_data)

    processed_data: pd.DataFrame = process_by_time_range(processed_data, time_period)
    processed_data: pd.DataFrame = process_by_port(processed_data, ports, g_src_dst)
    processed_data: pd.DataFrame = process_by_protocol(processed_data, protocol)
    processed_data: pd.DataFrame = process_by_ip_range(processed_data, g_src_dst, ipv4_range_str.split(','), ipv6_range_str.split(','), single_or_pair)

    processed_data: pd.DataFrame = process_by_type(processed_data, g_src_dst, g_data_type, top_n, single_or_pair, minimal_duration)
    
    processed_data.sort_values(processed_data.columns[1])
        
    return processed_data

def on_change_single_or_pair(v):
    if single_or_pair_widget.value == 'Pair':
        disable_button(src_dst_widget)
    else:
        enable_button(src_dst_widget)

def on_change_data_type(v):
    clear_output(wait=True)
    data_type = data_type_widget.value

    if data_type == 'Duration with dst IP':
        minimal_duration_widget.disabled = False
        minimal_duration_widget.value = "5"
        disable_button(src_dst_widget)
        single_or_pair_widget.value = 'Single'
        disable_button(single_or_pair_widget)

    elif data_type in ['IP requesting unusual TLD',
                        'Subdomains of muni.cz',
                        'Unusual TLDs', 
                        'Most DNS requests']: 
        minimal_duration_widget.disabled = True
        minimal_duration_widget.value = "disabled"
        disable_button(src_dst_widget)
        single_or_pair_widget.value = 'Single'
        disable_button(single_or_pair_widget)
    
    elif single_or_pair_widget.value == 'Single' or single_or_pair_widget.disabled:
            enable_button(src_dst_widget)
            enable_button(single_or_pair_widget)
    else: 
        disable_button(src_dst_widget)

    if data_type != 'Duration with dst IP':
        minimal_duration_widget.disabled = True
        minimal_duration_widget.value = "disabled"


def make_graph(g_data_type, top_n, graph_data):
    tooltips = [g_data_type]

    if g_data_type in ["BYTES", "PACKETS", "DURATION", 
                       "DNS QNAME", "UNUSUAL TLD"]:
        tooltips.append("COUNT")

    brush = alt.selection_interval(name="brush")
    graph = alt.Chart(graph_data).mark_bar().encode(
        x=graph_data.columns[1],
        y=alt.Y(graph_data.columns[0], sort='-x'),
        tooltip=tooltips,
        color=alt.Color(graph_data.columns[1],
                   scale=alt.Scale(range=['lightgreen', 'green']))
    ).properties(
        width=600,
        height=1000 if top_n >= 30 else 600,
        autosize=alt.AutoSizeParams(
            type='fit',
            contains='padding'
        ),
    ).add_params(brush)
    
    return alt.JupyterChart(graph)
    
def on_select_ip_direction(change):
    if src_dst_widget.value == "Received":
        on_select_src_ip_address(change)
    else:
        on_select_dst_ip_address(change)

def on_select_src_ip_address(change):
    sel = change.new.value
    if sel is None or SRC_IP not in sel:
        filtered = flow_data.iloc[:0]
    else:
        filtered = flow_data[flow_data[SRC_IP].isin(sel[SRC_IP])]
        filtered = process_by_port(filtered, port_widget.value, "Received")
        filtered = process_by_protocol(filtered, protocol_widget.value)

    table_widget.value = filtered.to_html()

def on_select_dst_ip_address(change):
    sel = change.new.value
    if sel is None or DST_IP not in sel:
        filtered = flow_data.iloc[:0]
    else:
        filtered = flow_data[flow_data[DST_IP].isin(sel[DST_IP])]
        filtered = process_by_port(filtered, port_widget.value, "Sent")
        filtered = process_by_protocol(filtered, protocol_widget.value)

    table_widget.value = filtered.to_html()

def on_select_src_and_dst_ip_address(change):
    sel = change.new.value
    if sel is None or "pair" not in sel:
        ret_df = flow_data.iloc[:0]
    else:
        ret_df = pd.DataFrame()

        for i in range(len(sel["pair"])):
            filtered = flow_data[((flow_data[SRC_IP] == sel["pair"][i][0]) & (flow_data[DST_IP] == sel["pair"][i][1]))  |
                            (flow_data[DST_IP] == sel["pair"][i][0]) & (flow_data[SRC_IP] == sel["pair"][i][1])]
            ret_df = pd.concat([ret_df, filtered])

        ret_df = process_by_port(ret_df, port_widget.value, "Sent")
        ret_df = process_by_protocol(ret_df, protocol_widget.value)

    table_widget.value = ret_df.to_html()

def on_select_unusual_tld(change):
    sel = change.new.value
    if sel is None or "UNUSUAL TLD" not in sel:
        filtered = flow_data.iloc[:0]
    else:
        filtered = flow_data[flow_data["DNS QNAME"].isin(sel["UNUSUAL TLD"])]
        filtered = process_by_port(filtered, port_widget.value, "Sent")
        filtered = process_by_protocol(filtered, protocol_widget.value)

    table_widget.value = filtered.to_html()

def on_select_muni_subdomain(change):
    sel = change.new.value
    if sel is None or "DNS QNAME" not in sel:
        filtered = flow_data.iloc[:0]
    else:
        filtered = flow_data[flow_data["DNS QNAME"].isin(sel["DNS QNAME"])]
        filtered = process_by_port(filtered, port_widget.value, "Sent")
        filtered = process_by_protocol(filtered, protocol_widget.value)

    table_widget.value = filtered.to_html()

def on_change(v):

    clear_output(wait=True)
    display_widgets()

    on_select_func = None
    data_type_key = data_type_widget.value
    single_pair_key = single_or_pair_widget.value

    g_src_dst = SRC_IP if src_dst_widget.value == "Received" else DST_IP
    
    if (DATA_TYPE_DSC[data_type_key] in ["BYTES", "PACKETS", 
                                         "FLOWS", "IP REQUESTING UNUSUAL TLD",
                                         "MOST DNS REQUESTS"] 
        and SINGLE_PAIR_DSC[single_pair_key] == "SINGLE"):
        on_select_func = on_select_ip_direction
    elif DATA_TYPE_DSC[data_type_key] == "UNUSUAL TLD":
        on_select_func = on_select_unusual_tld
    elif DATA_TYPE_DSC[data_type_key] == "DURATION":
        on_select_func = on_select_dst_ip_address
    elif DATA_TYPE_DSC[data_type_key] == "DNS QNAME":
        on_select_func = on_select_muni_subdomain
    elif SINGLE_PAIR_DSC[single_pair_key] == "PAIR":
        on_select_func = on_select_src_and_dst_ip_address

    graph_data = get_traffic_data(g_src_dst, DATA_TYPE_DSC[data_type_key],
                            int(top_n_widget.value), period_widget.value,
                            ipv4_range_widget.value,ipv6_range_widget.value,
                            single_or_pair_widget.value, protocol_widget.value,
                            port_widget.value, minimal_duration_widget.value)

    if graph_data.empty:
        print("Log: The data set is empty")
        return
    graph: alt.JupyterChart = make_graph(DATA_TYPE_DSC[data_type_key], 
                                         int(top_n_widget.value), 
                                         graph_data)

    if (on_select_func):
        graph.selections.observe(on_select_func, ["brush"])

    display(widgets.VBox([graph, table_widget]))


single_or_pair_widget.on_trait_change(on_change_single_or_pair)
data_type_widget.on_trait_change(on_change_data_type)
apply_changes_widget.on_click(on_change)
display_widgets()



Dropdown(description='Top N stats', options=('10', '20', '30', '40'), style=DescriptionStyle(description_width…

ToggleButtons(button_style='info', description='Info about single IP addresss or pair:', options=('Single', 'P…

ToggleButtons(button_style='info', description='Direction:', options=('Received', 'Sent'), style=ToggleButtons…

ToggleButtons(button_style='info', description='flow_data type:', index=8, layout=Layout(width='520px'), optio…

Text(value='disabled', description='Minimal duration of a flow in seconds', disabled=True, layout=Layout(width…

...

SelectMultiple(description='Protocol', index=(0,), options=('ALL', 'TCP', 'UDP', 'SMTP', 'DNS', 'HTTP', 'HTTPS…

Dropdown(description='Ports', options=('ALL', '8080, 8888, 591, 82', '8443, 9443, 4443', '5353, 5355', '2121, …

Button(button_style='success', description='Apply changes', style=ButtonStyle(), tooltip='Apply changes')

('ALL',)


error: nothing to repeat at position 0